<div style="background-color:#f0f8ff; border-radius:8px; padding:12px; text-align:center;">

# **PRACTICA 2: Árboles y Ensembles**

</div>

*Aprendizaje Automático*

---

**Grupo:** G-7312  
**Número de pareja:** 01  
**Miembros:**  
- Leire Bernárdez Vázquez  
- Carmen Reiné Rueda

---

### **Importaciones**

In [27]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.pipeline import Pipeline
from sklearn.datasets import fetch_covtype
from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_selection import SelectFromModel

---
**1. Cargue los datos del problema**

In [28]:
# Load the Covertype dataset
cov_type = fetch_covtype()
# Separate the features and the target variable
X = cov_type.data
y = cov_type.target

**2. Separe los datos en training (50 %) y test (50 %)**

In [29]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.5, random_state=42, stratify=y
)

No hace falta escalarlo, lo dice la teacher (los árboles se tragan todo lo que les des)

**3. Utilice la clase DecisionTreeClassifier de Sklearn para resolver el problema
y dé un resultado de test usando la m ́etrica que considere oportuna**

In [30]:
decision_tree = DecisionTreeClassifier()
decision_tree.fit(X_train, y_train)
y_pred = decision_tree.predict(X_test)
porcentaje = accuracy_score(y_test, y_pred)
print("Accuracy:", porcentaje*100, "%")

Accuracy: 92.35816127721975 %


**4. ¿Había missing values en nuestro problema? ¿Qué efecto tendría en los árboles no completar los missing values ?**

In [31]:
np.isnan(X_train).sum()==0

True

Si existieran y no se imputaran, DecisionTreeClassifier daría error al entrenar, porque los árboles no soportan missing values; requieren que todos los datos estén completos para funcionar. Para solucionar este problema, se podrían imputar con media, mediana o un valor concreto.

**5. ¿Con qué parémetro podría controlarse la profundidad del ́arbol? ¿Qué ocurre si un ́arbol se inicializa sin dar un valor concreto a ese parámetro? ¿Crees que sería una buena práctica no darle ningún valor?**

a) La profundidad máxima del árbol se controla con el parámetro max_depth directamente, que por defecto es max_depth=None. Sin embargo, también se puede controlar indirectamente con min_samples_split, que no divide un nodo si tiene menos muestras que las requeridas, y con min_samples_leaf, que obliga a que cada hoja tenga un mínimo de muestras para evitar que existan ramas muy profundas con pocas muestras. Por lo tanto, con estos valores se reduce la profundidad máxima alcanzada.

b) Crecería tanto como datos se le pasaran sin ningún tipo de límite.

c) No, debido a que sin un límite establecido, los árboles grandes consumen más memoria, tardan más en entrenar y tienden a sobreajustarse más fácilmente. Con lo cual, sería importante definir un max_depth razonable teniendo en cuenta los datos.

**6. ¿Son sensibles los ́arboles al desequilibrio entre clases?**

Sí, los árboles de decisión se ven afectados cuando las clases están desbalanceadas. Tratan de que cada hoja tenga solo un tipo de clase, así que cuando una clase es mayoritaria, las divisiones tienden a favorecerla. Esto puede hacer que las clases minoritarias queden mezcladas o se clasifiquen mal.

Si no se corrige este desequilibrio, el árbol aprenderá principalmente la clase mayoritaria y puede fallar al predecir las clases minoritarias, aunque acierte mucho con la clase que domina.


**7. ¿Existe desequilibrio entre las clases de nuestro problema? ¿Con qué parámetro de la clase DecisionTreeClassifier podrías corregirlo?**

In [32]:
counts = np.bincount(y)  # y debe ser enteros >=0
print({i: count for i, count in enumerate(counts) if count > 0})

{1: 211840, 2: 283301, 3: 35754, 4: 2747, 5: 9493, 6: 17367, 7: 20510}


a) Para comprobar si existe desequilibrio entre las clases del dataset Covtype, hemos utilizado la función np.bincount sobre las etiquetas de cada muestra.  

   Esta función nos permite contar cuántas muestras hay de cada clase y así analizar la distribución de los datos. Los resultados muestran que algunas clases tienen muchas más muestras que otras, confirmando que sí existe desequilibrio entre las clases.  

b) Podríamos corregirlo utilizando el parámetro class_weight='balanced  en DecisionTreeClassifier, que ajusta automáticamente los pesos de cada clase en función de su frecuencia en el dataset.

**8. ¿Cuál es la diferencia entre los parámetros min samples split y min samples leaf ?**

 **`min_samples_split`** 

**Definición según la documentación:** El número mínimo de muestras necesarias para dividir un nodo interno.

Este parámetro decide si un nodo se puede dividir. 
- Si el grupo de datos tiene suficientes muestras según el valor indicado, el nodo se divide en dos o más partes.  
- Si no tiene suficientes, el nodo se queda igual y se convierte en hoja.


**`min_samples_leaf`**  

**Definición según la documentación:** El número mínimo de muestras necesarias para estar en un nodo hoja. Un punto de división a cualquier profundidad solo se considerará si sale a menos muestras de entrenamiento en cada una de las muestras de entrenamiento izquierdo y ramas derechas. Esto puede tener el efecto de suavizar el modelo

Este parámetro se utiliza después de dividir un nodo para comprobar si cada hoja tiene suficientes muestras.  
- Si alguna hoja tiene menos muestras que el valor indicado, la división no se realiza.


**9. ¿Para qué sirve el parémetro criterion, qué valores puede tomar y qué significa o representa cada uno de esos valores?**

El parámetro criterion sirve para decidir qué tan buena es una división en un árbol de decisión. Es decir, indica cómo medir la calidad de los splits.

Los valores que puede tomar son:

- **gini** → utiliza el índice de Gini, que mide la impureza de un nodo.  
  Busca que cada hoja tenga muestras de la misma clase lo más posible.  

- **entropy** → utiliza la ganancia de información de Shannon.  
  Calcula cuánto disminuye la incertidumbre o desorden al hacer un split.  
  
- **log_loss** → también mide la ganancia de información, pero usando pérdida logarítmica.

**10. El ́arbol entrenado, ¿realiza splits utilizando todas las variables disponibles?**

In [33]:
importances = decision_tree.feature_importances_
used_features = [(i, imp) for i, imp in enumerate(importances) if imp > 0]
print("Variables usadas con importancia:", used_features)


Variables usadas con importancia: [(0, 0.3387193675902033), (1, 0.02734065294627229), (2, 0.017110056184974496), (3, 0.060196787910735905), (4, 0.047588420293639556), (5, 0.14774537629887685), (6, 0.03182038149798011), (7, 0.033703739434048204), (8, 0.02477366924852275), (9, 0.144395346998855), (10, 0.009571813426162762), (11, 0.0023373991008695996), (12, 0.012507758498829688), (13, 0.0016874797520611059), (14, 0.00029190964461878833), (15, 0.009824746700157672), (16, 0.001370541747133486), (17, 0.01125754414967012), (18, 0.0002653376729426192), (19, 0.0005795223900614763), (22, 0.00011716541185293037), (23, 0.002609083803495816), (24, 0.0018839788104798813), (25, 0.0008602858986689834), (26, 0.0034171015516396533), (27, 0.00011945702525846545), (28, 1.0886440090901796e-05), (29, 0.000779219015968397), (30, 0.0012164764635362703), (31, 1.8129399786179915e-05), (32, 0.0009461197934352617), (33, 0.002977966855033216), (34, 0.0002886730699214571), (35, 0.00694562633474603), (36, 0.0087649

Cada tupla (índice, importancia) indica qué columna se utilizó en algún split del árbol y cuánto aportó a la decisión de dividir los nodos.

Observando las importancias de las variables de nuestro árbol:

- No todas las variables se utilizan en los splits.  
- Solo se usan aquellas que aportan información útil para separar las clases, mientras que las demás quedan con importancia 0 y no aparecen en ningún nodo.  
- Aunque el árbol analiza todas las variables disponibles, solo mantiene en el modelo final las que realmente ayudan a mejorar la clasificación.



**11. ¿Se le ocurre alguna forma de crear un selector de atributos a partir de un árbol de clasificación?**

Aprovechando la importancia de cada variable feature_importances, podemos utilizar el árbol de decisión como selector de atributos.

- Manteniendo únicamente las variables que realmente aportan información al modelo.  
- Las variables con importancia 0 o muy baja pueden eliminarse, ya que no influyen en los splits.  
- Esto permite reducir la dimensionalidad, simplificar el modelo e incluso en algunos casos mejorar la generalización.

En Scikit-Learn se puede hacer fácilmente con SelectFromModel usando el árbol entrenado `decision_tree´.


In [ ]:
selector = SelectFromModel(decision_tree, threshold=0.01, prefit=True)
X_selected = selector.transform(X)

print(f"Número de variables originales: {X.shape[1]}")
print(f"Número de variables seleccionadas: {X_selected.shape[1]}")


Número de variables originales: 54
Número de variables seleccionadas: 13


SelectFromModel mira decision_tree.feature_importances_ y compara cada valor con  threshold = 0,01.  
Si la importancia es mayor o igual al threshold, se mantiene; si es menor, se descarta.  
Al hacer X_selected = selector.transform(X) conservamos solo las columnas importantes.



### **2. Regresión mediante árboles de regresión**

#### **2.1. Obtención de los datos**

**Descargue los datos de Sklearn**

In [ ]:
from sklearn.datasets import fetch_california_housing
data = fetch_california_housing()
X, y = data.data, data.target

**Separe los datos en training (50 %) test (50 %) y realice el preprocesado de los
datos que estime conveniente teniendo en cuenta que el modelo de aprendizaje con
el que va a tratar de resolver el problema es un  ́arbol de regresión. Finalmente
responda a las siguientes preguntas**

**1. ¿Podría trabajar un arbol de regresión con missing values?**

**2. ¿Es sensible un ́arbol de regresión a las magnitudes de los atributos?**

**3. ¿Es sensible un árbol de regresión a la distribución de las etiquetas?**

#### **2.2 Implementación de un árbol de regresión**



**1. Construya una clase Nodo**

**2. Construya la clase RegressionTree**

#### **2.3. Nos enfrentamos al problema de California Housing**

**1. ¿Cómo evoluciona el error de training y test conforme se aumenta la profundidad del  ́arbol? Base su razonamiento en dos gráficas obtenidas con matplotlib.**

**2. ¿Sobre qué parámetros tendría sentido realizar cross-validation en nuestro modelo?**


**3. Tome un par de muestras y utilizando el método decision_path(x) justifique la predicción dada. Como habrá comprobado, el método decision_path(x) devuelve una lista ordenada de argumentos, suponga que esa lista se desordena: ¿seguiría siendo válida la interpretación de la predicción?**

**4. Para una muestra idéntica a la del apartado anterior, si se entrenara un nuevo árbol, ¿la justificación de su predicción sería la misma?**

**5. Suponga que divide el conjunto de train en dos mitades, entrena dos árboles distintos, uno con cada mitad, y toma una muestra de test. ¿Sería la predicción de la muestra igual en cada árbol? ¿Y la interpretación obtenida con decision_path(x)? ¿Qué repercusión en la interpretación tendría esa situación?**


**6. Utilice la clase DecisionTreeRegressor de Sklearn para resolver de nuevo el problema. Compare el score de ambos modelos y, para alguna muestra, el resultado de decision_path(x) de ambos árboles.**

**7. En base a su implementación ¿cuál es el coste computacional teórico de entrenar un árbol? ¿Y cuál sería el coste en predicción? ¿Y el coste en memoria? Si tuviera que resolver un problema con un conjunto de datos de gran volumen, ¿qué preferiría usar, un árbol o un modelo de regresión lineal?**


### **3. Ensembles**